<a href="https://colab.research.google.com/github/moscadaa12/Construccion-de-Sotfwar/blob/main/modelo2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==========================================================
# 1. Importar librerías necesarias
# ==========================================================
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

print("Librerías importadas correctamente.")


Librerías importadas correctamente.


In [ ]:
# ==========================================================
# 2. Cargar y preparar el conjunto de datos
# ==========================================================

# URL pública del conjunto de datos de Diabetes de los indios Pima
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv"

# Cargar los datos (sin encabezados)
df = pd.read_csv(url, header=None)

# Asignar nombres de columnas
df.columns = ['Embarazos', 'Glucosa', 'PresiónArterial', 'GrosorPiel', 'Insulina',
              'IMC', 'FuncionPedigreeDiabetes', 'Edad', 'Resultado']

print("Conjunto de datos cargado exitosamente.")
display(df.head())


Conjunto de datos cargado exitosamente.


,Embarazos,Glucosa,PresiónArterial,GrosorPiel,Insulina,IMC,FuncionPedigreeDiabetes,Edad,Resultado
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
# ==========================================================
# 3. Separar variables y escalar datos
# ==========================================================

# Variables independientes (X) y dependiente (y)
X = df.drop('Resultado', axis=1)
y = df['Resultado']

# Dividir en datos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Escalar los datos (normalización)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Datos divididos y escalados exitosamente.")
print(f"Forma de X_train_scaled: {X_train_scaled.shape}")
print(f"Forma de X_test_scaled: {X_test_scaled.shape}")


Datos divididos y escalados exitosamente.
Forma de X_train_scaled: (537, 8)
Forma de X_test_scaled: (231, 8)


In [ ]:
# ==========================================================
# 4. Definir la función de activación (escalón)
# ==========================================================

def funcion_escalon(suma_ponderada):
    """Devuelve 1 si la suma ponderada es positiva o 0 si es negativa."""
    return 1 if suma_ponderada >= 0 else 0

print("Función escalón definida correctamente.")


Función escalón definida correctamente.


In [ ]:
# ==========================================================
# 5. Crear la clase Perceptrón Modificado (con decaimiento de tasa de aprendizaje)
# ==========================================================

class PerceptronModificado:
    """
    Perceptrón simple que implementa una tasa de aprendizaje con decaimiento progresivo.
    """

    def __init__(self, tasa_inicial=0.5, decaimiento=0.02, epocas=100):
        """
        Inicializa los parámetros del modelo.
        Args:
            tasa_inicial (float): Tasa de aprendizaje inicial.
            decaimiento (float): Parámetro que controla cuánto disminuye la tasa con las épocas.
            epocas (int): Número de iteraciones de entrenamiento.
        """
        self.tasa_inicial = tasa_inicial
        self.decaimiento = decaimiento
        self.epocas = epocas
        self.pesos = None
        self.sesgo = None

    def _tasa_actual(self, epoca):
        """Calcula la tasa de aprendizaje actual según la época."""
        return self.tasa_inicial / (1 + self.decaimiento * epoca)

    def entrenar(self, X, y):
        """
        Entrena el modelo usando la regla del perceptrón y tasa de aprendizaje variable.
        """
        n_muestras, n_caracteristicas = X.shape

        # Inicializamos pesos y sesgo en cero
        self.pesos = np.zeros(n_caracteristicas)
        self.sesgo = 0

        # Repetimos el proceso por el número de épocas
        for epoca in range(self.epocas):
            tasa_actual = self._tasa_actual(epoca)
            # Recorremos cada muestra del conjunto
            for i, x_i in enumerate(X):
                # Calculamos la suma ponderada
                salida_lineal = np.dot(x_i, self.pesos) + self.sesgo

                # Aplicamos la función escalón
                y_pred = funcion_escalon(salida_lineal)

                # Calculamos el error
                error = y[i] - y_pred

                # Actualizamos los pesos y el sesgo
                self.pesos += tasa_actual * error * x_i
                self.sesgo += tasa_actual * error

            # Mostrar la tasa actual cada 20 épocas para observar su cambio
            if (epoca + 1) % 20 == 0:
                print(f"Época {epoca + 1}/{self.epocas} - Tasa actual: {tasa_actual:.4f}")

    def predecir(self, X):
        """Realiza predicciones (0 o 1) usando los pesos aprendidos."""
        salida_lineal = np.dot(X, self.pesos) + self.sesgo
        y_pred = np.array([funcion_escalon(s) for s in salida_lineal])
        return y_pred

print("Clase PerceptronModificado definida correctamente.")


Clase PerceptronModificado definida correctamente.


In [ ]:
# ==========================================================
# 6. Crear una instancia del modelo y entrenarlo
# ==========================================================

modelo_modificado = PerceptronModificado(tasa_inicial=0.5, decaimiento=0.02, epocas=100)

modelo_modificado.entrenar(X_train_scaled, y_train.to_numpy())

print("Modelo Perceptrón Modificado entrenado exitosamente.")


Época 20/100 - Tasa actual: 0.3623
Época 40/100 - Tasa actual: 0.2809
Época 60/100 - Tasa actual: 0.2294
Época 80/100 - Tasa actual: 0.1938
Época 100/100 - Tasa actual: 0.1678
Modelo Perceptrón Modificado entrenado exitosamente.


In [ ]:
# ==========================================================
# 7. Evaluar el desempeño del modelo
# ==========================================================
from sklearn.metrics import accuracy_score, confusion_matrix

# 1. Obtener predicciones del conjunto de prueba
y_pred = modelo_modificado.predecir(X_test_scaled)

# 2. Calcular precisión (accuracy)
precision = accuracy_score(y_test, y_pred)

# 3. Calcular matriz de confusión
matriz_confusion = confusion_matrix(y_test, y_pred)

# 4. Calcular error de clasificación
error_clasificacion = 1 - precision

# 5. Mostrar resultados
print(f"\nPrecisión del modelo: {precision:.4f}")
print(f"Error de Clasificación: {error_clasificacion:.4f}")
print("\nMatriz de Confusión:")
print(matriz_confusion)



Precisión del modelo: 0.7619
Error de Clasificación: 0.2381

Matriz de Confusión:
[[124  27]
 [ 28  52]]


In [ ]:
# ==========================================================
# 8. Mostrar resultados con etiquetas claras
# ==========================================================

print("--- Resultados finales del Perceptrón 2 (Modificado) ---")

matriz_confusion_df = pd.DataFrame(
    matriz_confusion,
    index=['Negativo Real', 'Positivo Real'],
    columns=['Negativo Predicho', 'Positivo Predicho']
)

display(matriz_confusion_df)

print(f"\nPrecisión del modelo: {precision:.4f}")
print(f"Error de Clasificación: {error_clasificacion:.4f}")


--- Resultados finales del Perceptrón 2 (Modificado) ---


,Negativo Predicho,Positivo Predicho
Negativo Real,124,27
Positivo Real,28,52



Precisión del modelo: 0.7619
Error de Clasificación: 0.2381
